In [52]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [53]:
using JuMP, GLPK
using DataFrames
using CSV

In [54]:
# Load the data
scenarios_200 = CSV.read("../data/scenarios_200.csv", DataFrame)
# multiply grid excess column with 1.0


# create a dictonary with 200 dataframes for each scenario
scenarios = Dict()
for i in 1:200
    df_helper = DataFrame(scenarios_200[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    #set column names
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    #change grid excess column to float
    
    scenarios[i] = df_helper
end

In [55]:
scenarios[1]

Row,price,wind power,grid_excess
,Float64,Float64,Float64
1,122.83,151.992,1.0
2,132.91,143.459,1.0
3,132.9,142.587,0.0
4,125.03,146.618,1.0
5,120.07,150.136,1.0
6,139.83,151.102,1.0
7,164.23,145.927,0.0
8,186.66,143.261,1.0
9,186.95,142.279,0.0


In [56]:
scenarios[2]

Row,price,wind power,grid_excess
,Float64,Float64,Float64
1,2.01,22.3372,1.0
2,1.38,18.6099,1.0
3,0.09,15.9499,0.0
4,0.08,13.4696,1.0
5,0.05,9.2429,1.0
6,0.0,5.90827,1.0
7,-0.38,4.51948,0.0
8,0.0,2.73391,1.0
9,0.0,1.1039,0.0


In [57]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])




Row,hour,p_DA
,Int64,Float64


In [58]:
scenarios[1][2,"price"]


132.91

In [59]:
W = 200
hours = 24

24

In [60]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])
#@variable(model, delta_up[1:W,1:hours])
#@variable(model, delta_down[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W .* (scenarios[i][hour,"price"] * p_DA[hour] + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
#@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
#@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
#@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)



# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

Optimal solution found
Objective value: 147329.6325903077


In [61]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))
#println("delta_up: ", value.(delta_up))
#println("delta_down: ", value.(delta_down))

Objective value: 147329.6325903077
p_DA: [0.0, 200.0, 0.0, 200.0, 200.0, 0.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 200.0, 200.0, 0.0, 0.0]
delta: [151.99241932258064 -56.54080645161292 142.5867741935484 -53.381612919354836 -49.864032306451634 151.10161283870966 145.9269354516129 143.26112901612905 -57.72129037096775 142.21532251612905 141.57870962903226 -61.68225808064517 137.33516127419355 136.8064515967742 132.9970967419355 129.52919346774195 121.8299999516129 115.12596767741935 108.50145154838708 103.54096774193548 -101.8177419516129 -107.83403232258064 84.6527419032258 78.06951611290323; 22.33724183870968 -181.39012912903226 15.949935451612903 -186.53041943548388 -190.75709683870969 5.90827414516129 4.51948385483871 2.7339112741935483 -198.89610322580646 0.9774112741935485 0.7919354838709678 -199.2580693548387 0.9741935483870968 1.167741935483871 1.3273499999999998 2.3072451612903224 3.326635483870968 4.499983870967742 5.493548387096774 5.038725